In [5]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.circuit.library import MCMT

# import basic plot tools
from qiskit.visualization import plot_histogram

# Load IBM Q account and get the least busy backend device
provider = IBMQ.load_account()
# provider = IBMQ.get_provider("ibm-q")
provider = IBMQ.get_provider(hub='ibm-q-education', group='yale-uni-2', project='cpsc647-quantum')
# device = provider.get_backend('ibmq_lima') #run on ibmq_lima (5-qbit QC)

In [92]:
# M1. Series of CX (with optimization mode = 0) 
# M2. Series of CX+Delay[0] (with optimization mode on) 
# M3. Series of CX with alternating controls 
# M4. Series of CX+H+H 
# M5. Series of X 
# M6. Series of Y 
# M7. Series of Z 
# M8. Series of H 
# M9. Series of CX alternating between 3 Qubits 
# M10. Series of CX steps between 4 Qubits



def malicious_circuit_gen(copies, mal_type): # k- decides the depth of the malicious circuit and d is the delay value
    
    if (mal_type == 'M10'):
        n = 4
    
    elif (mal_type == 'M9'):
        n = 3
    
    elif (mal_type == 'M5' or mal_type == 'M6' or mal_type == 'M7' or mal_type == 'M8'  ):
        n = 1     
    
    else:
        n = 2
    mal_circuit = QuantumCircuit(n)
    
############ MALICIOUS CIRCUIT #####################
    for i in range(copies):
        if (mal_type == 'M1'):
            mal_circuit.cx(0,1)
        
        if (mal_type == 'M2'):
            mal_circuit.cx(0,1)
            mal_circuit.delay(0, qarg=0, unit = 'dt')
        
        if (mal_type == 'M3'):
            mal_circuit.cx(0,1)
            mal_circuit.delay(0, qarg=0, unit = 'dt')
            mal_circuit.cx(1,0)
            mal_circuit.delay(0, qarg=0, unit = 'dt')
        
        if (mal_type == 'M4'):
            mal_circuit.cx(0,1)
            mal_circuit.h(0)
            mal_circuit.delay(0, qarg=0, unit = 'dt')
            mal_circuit.h(0)
        
        if (mal_type == 'M5'):
            mal_circuit.x(0)
            mal_circuit.delay(0, qarg=0, unit = 'dt')
        
        if (mal_type == 'M6'):
            mal_circuit.y(0)
            mal_circuit.delay(0, qarg=0, unit = 'dt')
        
        if (mal_type == 'M7'):
            mal_circuit.z(0)
            mal_circuit.delay(0, qarg=0, unit = 'dt')
        
        if (mal_type == 'M8'):
            mal_circuit.h(0)
            mal_circuit.delay(0, qarg=0, unit = 'dt')
        
        if (mal_type == 'M9'):
            mal_circuit.cx(0,1)
            mal_circuit.cx(1,2)
        
        if (mal_type == 'M10'):
            if i%2==1:
                mal_circuit.cx(1,0)
                mal_circuit.cx(2,1)
                mal_circuit.cx(3,2)
                mal_circuit.delay(0, qarg = 3)
            else:
                mal_circuit.cx(3,2)
                mal_circuit.cx(2,1)
                mal_circuit.cx(1,0)
                mal_circuit.delay(0, qarg = 0)
####################################################

#     mal_circuit.barrier()
#     mal_circuit.measure_all()
    
    return mal_circuit

In [112]:

# Mtype = ['M1','M2','M3','M4','M5','M6','M7','M8','M9','M10']
# copies = 2
# i = 1
# for i in range(len(Mtype)):
mal_circuit = malicious_circuit_gen(1,'M3')
print(mal_circuit)
mal_circuit.draw('latex_source', filename='./M1.tex')
!pdflatex M1.tex

          ┌──────────────┐┌───┐┌──────────────┐
q_0: ──■──┤ Delay(0[dt]) ├┤ X ├┤ Delay(0[dt]) ├
     ┌─┴─┐└──────────────┘└─┬─┘└──────────────┘
q_1: ┤ X ├──────────────────■──────────────────
     └───┘                                     
This is pdfTeX, Version 3.141592653-2.6-1.40.23 (MiKTeX 21.10)
entering extended mode
(M1.tex
LaTeX2e <2021-06-01> patch level 1
L3 programming layer <2021-10-12>
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/latex/standalone\standalone.c
ls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/latex/tools\shellesc.sty)
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/generic/iftex\ifluatex.sty
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/generic/iftex\iftex.sty))
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/latex/xkeyval\xkeyval.sty
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/generic/xkeyval\xkeyval.tex
(C:\Users\sd982\AppData\Local\Programs\MiKTe

'M1'

In [83]:
def initialize_s(qc, qubits):
    """Apply a H-gate to 'qubits' in qc"""
    for q in qubits:
        qc.h(q)
    return qc

def grovers_algorithm(copies, mal_type): # k- decides the depth of the malicious circuit and d is the delay value
    
    if (mal_type == 'M10' or mal_type == 'M11'):
        n = 6
    
    elif (mal_type == 'M9'):
        n = 5
    elif (mal_type == 'M12' or mal_type == 'M0'):
        n = 2
    else:
        n = 4     
    
#     grover_circuit = QuantumCircuit(n)
    grover_circuit = QuantumCircuit(n,2)
    
#     grover_circuit.barrier()
############## VICTIM's CIRCUIT ######################    
    grover_circuit = initialize_s(grover_circuit, [0,1])
   
    grover_circuit.cz(0,1) # Oracle

    # Diffusion operator (U_s)
    grover_circuit.h([0,1])
    grover_circuit.z([0,1])
    grover_circuit.cz(0,1)
    grover_circuit.h([0,1])

####################################################

############ MALICIOUS CIRCUIT #####################
    if (mal_type == 'M12'):
        grover_circuit.barrier()
        grover_circuit.delay(10**copies, unit = 'dt')
    
    else:
        for i in range(copies):
            if (mal_type == 'M1'):
                grover_circuit.cx(2,3)

            if (mal_type == 'M2'):
                grover_circuit.cx(2,3)
                grover_circuit.delay(0, qarg=2, unit = 'dt')

            if (mal_type == 'M3'):
                grover_circuit.cx(2,3)
                grover_circuit.cx(3,2)

            if (mal_type == 'M4'):
                grover_circuit.cx(2,3)
                grover_circuit.h(2)
                grover_circuit.delay(0, qarg=2, unit = 'dt')
                grover_circuit.h(2)

            if (mal_type == 'M5'):
                grover_circuit.x(2)
                grover_circuit.delay(0, qarg=2, unit = 'dt')

            if (mal_type == 'M6'):
                grover_circuit.y(2)
                grover_circuit.delay(0, qarg=2, unit = 'dt')

            if (mal_type == 'M7'):
                grover_circuit.z(2)
                grover_circuit.delay(0, qarg=2, unit = 'dt')

            if (mal_type == 'M8'):
                grover_circuit.h(2)
                grover_circuit.delay(0, qarg=2, unit = 'dt')

            if (mal_type == 'M9'):
                grover_circuit.cx(2,3)
                grover_circuit.cx(3,4)

            if (mal_type == 'M10'):
                if i%2==1:
                    grover_circuit.cx(3,2)
                    grover_circuit.cx(4,3)
                    grover_circuit.cx(5,4)
                    grover_circuit.delay(0, qarg = 5)
                else:
                    grover_circuit.cx(5,4)
                    grover_circuit.cx(4,3)
                    grover_circuit.cx(3,2)
                    grover_circuit.delay(0, qarg = 2)

            if (mal_type == 'M11'):
                grover_circuit.cx(4,5)
                grover_circuit.delay(0, qarg=4, unit = 'dt')


            
####################################################

#     grover_circuit.barrier()
#     grover_circuit.measure_all()
    grover_circuit.measure(0,0)
    grover_circuit.measure(1,1)
    
    return grover_circuit

In [85]:
grover_circuit = grovers_algorithm(0,'M0')
print(grover_circuit)

grover_circuit.draw('latex_source', filename='./grovers_circuit.tex')
!pdflatex grovers_circuit.tex

     ┌───┐   ┌───┐┌───┐   ┌───┐┌─┐   
q_0: ┤ H ├─■─┤ H ├┤ Z ├─■─┤ H ├┤M├───
     ├───┤ │ ├───┤├───┤ │ ├───┤└╥┘┌─┐
q_1: ┤ H ├─■─┤ H ├┤ Z ├─■─┤ H ├─╫─┤M├
     └───┘   └───┘└───┘   └───┘ ║ └╥┘
c: 2/═══════════════════════════╩══╩═
                                0  1 
This is pdfTeX, Version 3.141592653-2.6-1.40.23 (MiKTeX 21.10)
entering extended mode
(grovers_circuit.tex
LaTeX2e <2021-06-01> patch level 1
L3 programming layer <2021-10-12>
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/latex/standalone\standalone.c
ls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/latex/tools\shellesc.sty)
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/generic/iftex\ifluatex.sty
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/generic/iftex\iftex.sty))
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/latex/xkeyval\xkeyval.sty
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/generic/xkeyval\xkeyval.tex
(C:\U

In [42]:
def dj_oracle(case, n):
    # We need to make a QuantumCircuit object to return
    # This circuit has n+1 qubits: the size of the input,
    # plus one output qubit
    oracle_qc = QuantumCircuit(n+1)
    
    # First, let's deal with the case in which oracle is balanced
    if case == "balanced":
        # First generate a random number that tells us which CNOTs to
        # wrap in X-gates:
        b = np.random.randint(1,2**n)
        # Next, format 'b' as a binary string of length 'n', padded with zeros:
        b_str = format(b, '0'+str(n)+'b')
        # Next, we place the first X-gates. Each digit in our binary string 
        # corresponds to a qubit, if the digit is 0, we do nothing, if it's 1
        # we apply an X-gate to that qubit:
        for qubit in range(len(b_str)):
            if b_str[qubit] == '1':
                oracle_qc.x(qubit)
        # Do the controlled-NOT gates for each qubit, using the output qubit 
        # as the target:
        for qubit in range(n):
            oracle_qc.cx(qubit, n)
        # Next, place the final X-gates
        for qubit in range(len(b_str)):
            if b_str[qubit] == '1':
                oracle_qc.x(qubit)

    # Case in which oracle is constant
    if case == "constant":
        # First decide what the fixed output of the oracle will be
        # (either always 0 or always 1)
        output = np.random.randint(2)
        if output == 1:
            oracle_qc.x(n)
    
    oracle_gate = oracle_qc.to_gate()
    oracle_gate.name = "Oracle" # To show when we display the circuit
    return oracle_gate


def dj_algorithm(oracle, n):
    dj_circuit = QuantumCircuit(n+1, n)
    # Set up the output qubit:
    dj_circuit.x(n)
    dj_circuit.h(n)
    # And set up the input register:
    for qubit in range(n):
        dj_circuit.h(qubit)
    # Let's append the oracle gate to our circuit:
    dj_circuit.append(oracle, range(n+1))
    # Finally, perform the H-gates again and measure:
    for qubit in range(n):
        dj_circuit.h(qubit)
    
    
    for i in range(n):
        dj_circuit.measure(i, i)
    
    return dj_circuit

In [65]:
n=2
oracle = dj_oracle("balanced",n)

dj = dj_algorithm(oracle, n)
# print(dj)
dj.draw('latex_source', filename='./dj_circuit.tex')
!pdflatex dj_circuit.tex

     ┌───┐     ┌─────────┐┌───┐┌─┐   
q_0: ┤ H ├─────┤0        ├┤ H ├┤M├───
     ├───┤     │         │├───┤└╥┘┌─┐
q_1: ┤ H ├─────┤1 Oracle ├┤ H ├─╫─┤M├
     ├───┤┌───┐│         │└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤2        ├──────╫──╫─
     └───┘└───┘└─────────┘      ║  ║ 
c: 2/═══════════════════════════╩══╩═
                                0  1 


AttributeError: 'Gate' object has no attribute 'draw'

In [48]:
def bv_algorithm(s, n):
    # We need a circuit with n qubits, plus one auxiliary qubit
    # Also need n classical bits to write the output to
    bv_circuit = QuantumCircuit(n+1, n)

    # put auxiliary in state |->
    bv_circuit.h(n)
    bv_circuit.z(n)

    # Apply Hadamard gates before querying the oracle
    for i in range(n):
        bv_circuit.h(i)

    # Apply barrier 
    bv_circuit.barrier()

    # Apply the inner-product oracle
    s = s[::-1] # reverse s to fit qiskit's qubit ordering
    for q in range(n):
        if s[q] == '0':
            bv_circuit.i(q)
        else:
            bv_circuit.cx(q, n)

    # Apply barrier 
    bv_circuit.barrier()

    #Apply Hadamard gates after querying the oracle
    for i in range(n):
        bv_circuit.h(i)
    

    # Measurement
    for i in range(n):
        bv_circuit.measure(i, i)
    
    return bv_circuit

In [50]:
bv = bv_algorithm('10', 2)
print(bv)
bv.draw('latex_source', filename='./bv_circuit.tex')
!pdflatex bv_circuit.tex

     ┌───┐      ░ ┌───┐ ░ ┌───┐┌─┐   
q_0: ┤ H ├──────░─┤ I ├─░─┤ H ├┤M├───
     ├───┤      ░ └───┘ ░ ├───┤└╥┘┌─┐
q_1: ┤ H ├──────░───■───░─┤ H ├─╫─┤M├
     ├───┤┌───┐ ░ ┌─┴─┐ ░ └───┘ ║ └╥┘
q_2: ┤ H ├┤ Z ├─░─┤ X ├─░───────╫──╫─
     └───┘└───┘ ░ └───┘ ░       ║  ║ 
c: 2/═══════════════════════════╩══╩═
                                0  1 
This is pdfTeX, Version 3.141592653-2.6-1.40.23 (MiKTeX 21.10)
entering extended mode
(bv_circuit.tex
LaTeX2e <2021-06-01> patch level 1
L3 programming layer <2021-10-12>
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/latex/standalone\standalone.c
ls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/latex/tools\shellesc.sty)
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/generic/iftex\ifluatex.sty
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/generic/iftex\iftex.sty))
(C:\Users\sd982\AppData\Local\Programs\MiKTeX\tex/latex/xkeyval\xkeyval.sty
(C:\Users\sd